## Index Tables Data Export Codes

- 1. tbl_unit
- 2. tbl_log_parameters

In [1]:
# import required libraries
import numpy as np
import pandas as pd
import os
from datetime import date
import sqlalchemy
import pymysql

import warnings
warnings.filterwarnings("ignore")

In [2]:
# load all the file names into a list (files in the LOgData folder)
os.getcwd()
_dir_path = '../../../dataExport/LOGdata'
raw_data = os.listdir(_dir_path)
raw_data

['log_2022_4_20.csv',
 'log_2022_4_21.csv',
 'log_2022_5_18.csv',
 'log_2022_5_3.csv']

In [4]:
# set file name
file_name = '..\..\..\dataExport\LOGdata\log_2022_5_18.csv'

In [5]:
# load data into a dataframe
log_rawDF = pd.read_csv(file_name, sep='*')

In [6]:
# extract uniques units and parameters from the log data 
# to enterthem into the index tables
def log_prm_unit_values(log_rawDF):
    log_rawDF = log_rawDF.dropna(subset=['Parameter'])
    log_parameter = log_rawDF['Parameter'].unique()
    tmp_1 = log_rawDF.query('Type != (1)')
    tmp_1 = tmp_1.query('Unit == ("FPC11", "FPC12", "FPC13", "FPC14", "FPC21", "FPC22", "FPC23", "FPC24", "FFU", "FPP", "Product", "scheduler")')
    log_unit = tmp_1['Unit'].unique()
    return ([log_unit, log_parameter])

tbl_values = log_prm_unit_values(log_rawDF)

#print(tbl_values[0])
#print(tbl_values[1])

In [2]:
# database connection with mariaDB using pymysql package
def dBCon_Maria():
    connection = pymysql.connect(host='localhost',
                            user='root',
                            password='password',
                            db='data_dashboard')
    return connection

# connection = dBCon_Maria()
# cursor=connection.cursor()

''' 
============================ Sample query ==============================
#### ------------- Reading data from database table to data frame ---------------------- 
connection = dBCon_Maria()
cursor=connection.cursor()
trial_no = 2
tbl_lookup = pd.read_sql("SELECT * FROM tbl_pbr_lookup WHERE instance = %s", connection, params=[trial_no])

============== Selecting data from the table and printing it ==============================
# just a function for test the db connection is working or not
connection = dBCon_Maria()
cursor=connection.cursor()
cursor.execute("SELECT unit_id, unit_name FROM tbl_unit")
for (unit_id, unit_name) in cursor:
    print("ID:", {unit_id}, "unit_name:", {unit_name})

==================== query to return the maximum value from the column ==================
# just a function for test the db connection is working or not
connection = dBCon_Maria()
cursor=connection.cursor()
cursor.execute("SELECT MAX(unit_id) FROM tbl_unit")
result = cursor.fetchone()
print(result)
print(result[0])
'''
print("\n")                                 

In [8]:
# Insert unique units into index table tbl_unit
def insrt_tbl_unit(log_unit):
    try:
        log_unit = log_unit.tolist()
        connection = dBCon_Maria()
        cursor=connection.cursor()
        query = 'INSERT IGNORE INTO tbl_unit (unit_name) VALUES (%s)'
        cursor.executemany(query, log_unit)
        connection.commit()
        connection.close()
    except Exception as e: print(e)

insrt_tbl_unit(tbl_values[0])

In [10]:
# Insert unique parameters into index table : tbl_log_parameters
def insrt_tbl_parameter(log_parameter):
    try:
        log_parameter = log_parameter.tolist()
        connection = dBCon_Maria()
        cursor=connection.cursor()
        query = 'INSERT IGNORE INTO tbl_log_parameters (prm_name) VALUES (%s)'
        cursor.executemany(query, log_parameter)
        connection.commit()
        connection.close()
    except Exception as e: print(e)

insrt_tbl_parameter(tbl_values[1])

## PBR lookup data preprocessing & DB export.

- Table       : tbl_pbr_lookup
- File name   : tbl_pbr_lookup.xlsx

Operations : 
1. Load the data 
2. populate unit and species column with DB tables primary key
3. Export data into the pbr_lookpup table

In [10]:
# set the file path
file_name = file_name = '..\..\..\dataExport\\tbl_pbr_lookup.xlsx'

In [11]:
# load the excel data sheet and display first 5 rows
pbr_lookup = pd.read_excel(file_name,sheet_name='Sheet1')
pbr_lookup

,instance,unit,species,start_date,end_date,remark,raw_file_name
0,1,FPC13,1,1629468000,1633939195,missing values from 2021-10-3 2:56:55 to 2021-...,_1_FPC13_ChCal_1.csv
1,2,FPC13,1,1634652000,1644488995,NaN,_2_FPC13_ChCal_2.csv
2,3,FPC13,11,1625227200,1628157595,NaN,_3_FPC13_ThalaPs.csv
3,4,FPC14,2,1614763800,1618392595,NaN,_4_FPC14_chamu.csv
4,5,FPC14,7,1625486400,1639564200,NaN,_5_FPC14_RhoSa.csv
5,6,FPC14,11,1618561800,1625228995,NaN,_6_FPC14_thalaps.csv
6,7,FPC21,2,1634914800,1646645395,NaN,_7_FPC21_chamu.csv
7,8,FPC21,9,1623051000,1626771600,NaN,_8_FPC21_Techu.csv
8,9,FPC22,3,1620216000,1634632200,NaN,_9_FPC22_DiaLut.csv
9,10,FPC23,7,1616427000,1626690600,NaN,_10_FPC23_Rhosa.csv


In [12]:
def dbConn_sqlAlch():
    try:
        database_username = 'root'
        database_password = 'password'
        database_ip       = '127.0.0.1:3306'
        database_name     = 'data_dashboard'
        database_connection = sqlalchemy.create_engine('mariadb+mariadbconnector://{0}:{1}@{2}/{3}'.
                                                    format(database_username, database_password, 
                                                            database_ip, database_name))
        return database_connection
    except Exception as e: print(e)


database_connection = dbConn_sqlAlch()
    

In [13]:
# check DB connection is fine and fetch unique uint ID
tbl_unit = pd.read_sql('SELECT unit_id, unit_name FROM tbl_unit', database_connection)
tbl_unit.head(3)



,unit_id,unit_name
0,2,FFU
1,8,FPC11
2,10,FPC12


In [14]:
# replace unit names with primary key value
pbr_lookup['unit'] = pbr_lookup['unit'].map(tbl_unit.set_index('unit_name')['unit_id'])
pbr_lookup


,instance,unit,species,start_date,end_date,remark,raw_file_name
0,1,6,1,1629468000,1633939195,missing values from 2021-10-3 2:56:55 to 2021-...,_1_FPC13_ChCal_1.csv
1,2,6,1,1634652000,1644488995,NaN,_2_FPC13_ChCal_2.csv
2,3,6,11,1625227200,1628157595,NaN,_3_FPC13_ThalaPs.csv
3,4,1,2,1614763800,1618392595,NaN,_4_FPC14_chamu.csv
4,5,1,7,1625486400,1639564200,NaN,_5_FPC14_RhoSa.csv
5,6,1,11,1618561800,1625228995,NaN,_6_FPC14_thalaps.csv
6,7,4,2,1634914800,1646645395,NaN,_7_FPC21_chamu.csv
7,8,4,9,1623051000,1626771600,NaN,_8_FPC21_Techu.csv
8,9,3,3,1620216000,1634632200,NaN,_9_FPC22_DiaLut.csv
9,10,5,7,1616427000,1626690600,NaN,_10_FPC23_Rhosa.csv


In [15]:
try:
    pbr_lookup.to_sql(con=database_connection, name='tbl_pbr_lookup', if_exists='append', index=False)
except Exception as e: print(e)

In [ ]:
'''
def db_connection():
    try: con = mariadb.connect( 
        user="root", 
        password="password", 
        host="localhost", 
        port=3306, 
        database="data_dashboard" 
        )

    except mariadb.Error as ex: 
        print(f"An error occurred while connecting to MariaDB: {ex}") 
        sys.exit(1) 

    return con.cursor(),con

#cur, con = db_connection()

# code to close the data base connection
def db_close(con):
    con.close()


# just a function for test the db connection is working or not
def test_query():
    cur, con = db_connection()
    cur.execute("SELECT unit_id, unit_name FROM tbl_unit")
    for (unit_id, unit_name) in cur:
        print("ID:", {unit_id}, "unit_name:", {unit_name})

    db_close(con)

test_query()

# Insert unique units into index table tbl_unit
def insrt_tbl_unit(log_unit):
    try:
        cur, con = db_connection()
        query = 'INSERT IGNORE INTO tbl_unit (unit_id, unit_name) VALUES (%s, %s)'
        tuples = pd.Series(log_unit)
        tuples = [(index+1, value) for index, value in tuples.items()]
        cur.executemany(query, tuples)
        con.commit()
        db_close(con)
    except Exception as e: print(e)

insrt_tbl_unit(tbl_values[0])

# Insert unique parameters into index table : tbl_log_parameters
def insrt_tbl_parameter(log_parameter):
    try:
        cur, con = db_connection()
        query = 'INSERT IGNORE INTO tbl_log_parameters (prm_id, prm_name) VALUES (%s, %s)'
        tuples = pd.Series(log_parameter)
        tuples = [(index+1, value) for index, value in tuples.items()]
        cur.executemany(query, tuples)
        con.commit()
        db_close(con)
    except Exception as e: print(e)

insrt_tbl_parameter(tbl_values[1])

'''
